# Clean Dataset for Model

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Clean data for modeling

pitch = pd.read_csv('../data/mlb-pitches.csv', index_col = [0])
pitch = pitch[['player_name', 'p_throws', 'pitch_type','release_speed', 'release_spin_rate', 'spin_axis', 
               'pfx_-x', 'pfx_z', 'bauer_units', 'effective_speed', 'release_pos_-x', 'release_pos_x', 
               'release_pos_z', 'release_extension', 'release_pos_y', 'plate_-x', 'plate_x', 'plate_z', 
               'type', 'balls','strikes', 'pitch_count', 'delta_run_exp', 'stand', 'bb_type', 'description', 
               'events', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'launch_speed_angle', 'woba_value', 
               'woba_denom', 'xba', 'xwoba', 'babip_value', 'iso_value', 'at_bat_number', 'pitch_number', 
               'inning', 'inning_topbot', 'home_score', 'away_score', 'post_home_score', 'post_away_score', 
               'on_1b', 'on_2b', 'on_3b', 'outs_when_up']].copy()

pitch.dropna(subset = ['pitch_type', 'release_speed', 'release_spin_rate', 'pfx_-x', 'pfx_z',
                       'release_extension', 'release_pos_-x', 'release_pos_z', 'delta_run_exp'], inplace = True)

#Rename some columns
col_dict = {
    'release_speed': 'velo',
    'release_spin_rate': 'spin_rate',
    'launch_speed': 'exit_velo',
}

pitch.rename(columns = col_dict, inplace = True)
pitch.to_csv('../data/model-data.csv')

In [3]:
# Run Expectany Table

# 2010-2015 Run Expectancy
matrix = [[0, 0, 0, 0, 0.481], [1, 0, 0, 0, 0.859], [0, 1, 0, 0, 1.100], [1, 1, 0, 0, 1.437], 
          [0, 0, 1, 0, 1.350], [1, 0, 1, 0, 1.784], [0, 1, 1, 0, 1.964], [1, 1, 1, 0, 2.292], 
          [0, 0, 0, 1, 0.254], [1, 0, 0, 1, 0.509], [0, 1, 0, 1, 0.664], [1, 1, 0, 1, 0.884], 
          [0, 0, 1, 1, 0.950], [1, 0, 1, 1, 1.130], [0, 1, 1, 1, 1.376], [1, 1, 1, 1, 1.541],
          [0, 0, 0, 2, 0.098], [1, 0, 0, 2, 0.224], [0, 1, 0, 2, 0.319], [1, 1, 0, 2, 0.429], 
          [0, 0, 1, 2, 0.353], [1, 0, 1, 2, 0.478], [0, 1, 1, 2, 0.580], [1, 1, 1, 2, 0.752]]

re = pd.DataFrame(matrix, columns = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up', 're'])
# re.head()
re.to_csv('../data/run_expectancy_table.csv')

count_re = pd.read_csv('../data/count-re.csv')
print(count_re.shape)
#count_re.head()

(24, 5)


In [4]:
# Add RV to model-data.csv

data = pd.read_csv('../data/model-data.csv', index_col = [0])

# Add Arm angle
arm_angle = data.groupby(['player_name', 'pitch_type'], as_index = False)['release_pos_x', 
                                                                          'release_pos_z'].mean()
arm_angle = pd.DataFrame(arm_angle)
arm_angle['adj'] = (arm_angle['release_pos_z'] - arm_angle['release_pos_x']) * 0.7
arm_angle['opp'] = abs(arm_angle['release_pos_x'])
arm_angle['hyp'] = np.sqrt((arm_angle['opp'] ** 2) + (arm_angle['adj'] ** 2))
arm_angle['arm_angle']=round(np.arccos(((arm_angle['adj'] ** 2 + arm_angle['hyp'] ** 2) - arm_angle['opp'] ** 2) 
                                         / (2 * (arm_angle['adj'] * arm_angle['hyp']))), 3) * 100
arm_angle['slot'] = ['overhead' if (x >= 0) & (x <= 40) else '3/4' if (x >= 41) & (x <= 60) else 'sidearm'
                     for x in arm_angle['arm_angle']]
arm_angle.drop(columns = ['release_pos_x', 'release_pos_z', 'adj', 'opp', 'hyp'], inplace = True)
data = pd.merge(data, arm_angle, how = 'left', on = ['player_name', 'pitch_type'])

data = data[data['events'] != 'wild_pitch']
data = data[data['events'] != 'passed_ball']
data = data[data['events'] != 'stolen_base_2b']

data['events'].replace(['sac_bunt', 'double_play', 'caught_stealing_2b', 'strikeout_double_play',
                        'other_out', 'sac_fly_double_play', 'pickoff_2b', 'pickoff_3b', 'triple_play'
                        'caught_stealing_home', 'pickoff_caught_stealing_2b', 'pickoff_3b', 
                        'sac_bunt_double_play', 'pickoff_caught_stealing_3b', 'pickoff_1b', 
                        'caught_stealing_3b', 'triple_play', 'caught_stealing_home', 
                        'strikeout'], 'field_out', inplace = True)
data['events'].replace(['catcher_interf'], 'field_error', inplace = True)

data['description'].replace(['called_strike', 'swinging_strike', 'swinging_strike_blocked', 'missed_bunt',
                             'foul'], 'strike', inplace = True)
data['description'].replace(['passed_ball', 'wild_pitch'], 'ball', inplace = True)

data['is_strike'] = [1 if x != 'B' else 0 for x in data['type']]
data['is_ball'] = [1 if x == 'B' else 0 for x in data['type']]

data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['final_pitch_ab'] = [1 if x == x else 0 for x in data['events']]
data['out_to_end_inning'] = [1 if x == 'field_out' and y == 2 else 0 for (x, y) 
                             in zip(data['events'], data['outs_when_up'])]
data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

# Merge RE Table with data
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])
data['re_change'] = round(-data['re'].diff(1), 3)
data['re_change'].fillna(-0.098, inplace = True)
data['re_end_state'] = data['re'] + data['re_change']
data['re_end_state'] = [0 if x == 1 else y for (x, y) in zip(data['out_to_end_inning'], data['re_end_state'])]
data['re_change'] = [-y if x == 1 else z for (x, y, z) in zip(data['out_to_end_inning'], 
                                                              data['re'], data['re_change'])]
data['re_change'].replace([-0.000], 0.000, inplace = True)
data['re24'] = data['re_change'] + data['runs']

# Create Linear Weights with average RE by events
mlb_lw = data.groupby(['events'], as_index = False)['re24'].mean()
lw_ball_in_play = pd.DataFrame(mlb_lw)
lw_ball_in_play['re24'] = round(lw_ball_in_play['re24'], 3)
lw_ball_in_play.rename(columns = {'re24': 'lin_weight_above_avg'}, inplace = True)

# For lin weights based on base out state 
data = pd.merge(data, lw_ball_in_play, how = 'left', on = ['events'])
data['lin_weight_above_outs'] = data['lin_weight_above_avg'] + 0.25
data['woba_scale'] = 1.209
data['lin_weights_above_avg_scale'] = round(data['lin_weight_above_avg'] * data['woba_scale'], 3)
data['lin_weights_above_outs_scale'] = round(data['lin_weight_above_outs'] * data['woba_scale'], 3)

# Merge ball strike count RE with data
data = pd.merge(data, count_re, how = 'left', on = ['pitch_count', 'is_strike', 'is_ball'])
data['count_re'] = data['wraa_change'] + data['re24']

# # Create Linear Weights with average RE by ball strike count
mlb_lw_bs = data.groupby(['pitch_count'], as_index = False)['count_re'].mean()
lw_bs = pd.DataFrame(mlb_lw_bs)
lw_bs['count_re'] = round(lw_bs['count_re'], 3)
lw_bs.rename(columns = {'count_re': 'bs_lin_weight'}, inplace = True)
data = pd.merge(data, lw_bs, how = 'left', on = ['pitch_count'])
data['bs_lin_weight_scale'] = round(data['bs_lin_weight'] * data['woba_scale'], 3)

data['lin_weights_above_avg_scale'].fillna(0, inplace = True)
data['lin_weights_above_outs_scale'].fillna(0, inplace = True)

#data['lin_weights_above_avg_scale'].fillna(data['bs_lin_weight_scale'], inplace = True)
#data['lin_weights_above_outs_scale'].fillna(data['bs_lin_weight_scale'], inplace = True)
#data['lin_weights_above_outs_scale'] = [y if x == 0.000 else x for (x, y) in 
#                                        zip(data['lin_weights_above_outs_scale'], data['bs_lin_weight_scale'])]

#data['lin_weights_above_avg_scale'].fillna(data['bs_lin_weight'], inplace = True)
#data['lin_weights_above_outs_scale'].fillna(data['bs_lin_weight'], inplace = True)
#data['lin_weights_above_outs_scale'] = [y if x == 0.000 else x for (x, y) in 
#                                        zip(data['lin_weights_above_outs_scale'], data['bs_lin_weight'])]

data['rv_above_avg'] = data['lin_weights_above_avg_scale'] + data['bs_lin_weight_scale'] # * 100
data['rv'] = data['lin_weights_above_outs_scale'] + data['bs_lin_weight_scale'] # * 100)

data.to_csv('../data/model-pitches-rv.csv')

pd.set_option('max_columns', None)
print(data.shape)
data.head(10)

(705396, 74)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_-x,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,delta_run_exp,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwoba,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,arm_angle,slot,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_above_avg_scale,lin_weights_above_outs_scale,woba,wraa_change,count_re,bs_lin_weight,bs_lin_weight_scale,rv_above_avg,rv
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,-1.40,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,-0.073,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.0,1.0,0.174,0.158,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,36.0,overhead,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.302,0.0,0.223,-0.184,-0.282,-0.141,-0.170,-0.472,-0.170
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,-1.60,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,1-1,-0.027,R,NaN,strike,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,45.8,3/4,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.293,-0.058,-0.058,-0.006,-0.007,-0.007,-0.007
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,-1.46,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,1-0,-0.020,R,NaN,strike,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,38.4,overhead,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.355,-0.051,-0.051,0.001,0.001,0.001,0.001
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,-1.53,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0-0,0.016,R,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,38.4,overhead,0,1,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.314,0.034,0.034,-0.003,-0.004,-0.004,-0.004
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,-1.49,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,-0.189,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.0,1.0,0.100,0.090,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,36.0,overhead,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.302,0.0,0.355,-0.051,-0.207,0.001,0.001,-0.301,0.001
5,"Smith, Will",L,FF,92.5,2319.0,145.0,-6.48,17.40,25.070270,92.0,-1.35,1.35,6.74,6.3,54.19,1.04,-1.04,3.58,B,0,0,0-0,0.026,L,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,1,9,0,5,0,5,0,0,0,0,1,36.0,overhead,0,1,0,0,0,0,0,0.254,0.000,0.254,0.000,NaN,NaN,1.209,0.000,0.0,0.314,0.034,0.034,-0.003,-0.004,-0.004,-0.004
6,"Smith, Will",L,SL,80.3,2328.0,304.0,3.96,2.16,28.991283,80.8,-1.81,1.81,6.55,6.2,54.30,0.69,-0.69,0.83,S,0,2,0-2,-0.152,L,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,59,3,9,0,5,0,5,0,0,0,0,0,45.8,3/4,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.302,0.0,0.196,-0.162,-0.389,-0.123,-0.149,-0.451,-0.149
7,"Smith, Will",L,SL,79.5,2368.0,306.0,6.00,4.56,29.786164,79.4,-1.72,1.72,6.56,6.1,54.44,-0.12,0.12,1.99,S,0,1,0-1,-0.052,L,NaN,strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,2,9,0,5,0,5,0,0,0,0,0,45.8,3/4,1,0,0,0,0,0,0,0.481,0.000,0.481,0.000,NaN,NaN,1.209,0.000,0.0,0.262,-0.055,-0.055,-0.013,-0.016,-0.016,-0.016
8,"Smith, Will",L,FF,91.8,2299.0,146.0,-8.88,17.64,25.043573,91.8,-1.39,1.39,6.75,6.5,54.04,0.72,-0.72,2.77,S,0,0,0-0,-0.038,L,NaN,strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,1,9,0,5,0,5,0,0,0,0,0,36.0,overhead,1,0,0,0,0,0,0,0.481,0.000,0.481,0.000,NaN,NaN,1.209,0.000,0.0,0.314,-0.043,-0.043,-0.003,-0.004,-0.004,-0.004
9,"Gsellman, Robert",R,SI,94.3,1982.0,221.0,16.20,10.56,21.018028,94.4,1.20,-1.20,6.05,6.5,53.98,0.26,-0.26,1.78,X,0,2,0-2,-0.061,R,popup,hit_in

In [5]:
data2 = data.copy()
data2.dropna(subset = ['events'], inplace = True)
#data2.to_csv('../data/model-plate-app-rv.csv')

print(data2.shape)
data2.head(10)

(180159, 74)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_-x,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,delta_run_exp,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwoba,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,arm_angle,slot,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_above_avg_scale,lin_weights_above_outs_scale,woba,wraa_change,count_re,bs_lin_weight,bs_lin_weight_scale,rv_above_avg,rv
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,-1.40,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,-0.073,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.0,1.0,0.174,0.158,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,36.0,overhead,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.302,0.0,0.223,-0.184,-0.282,-0.141,-0.170,-0.472,-0.170
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,-1.49,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,-0.189,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.0,1.0,0.100,0.090,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,36.0,overhead,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.302,0.0,0.355,-0.051,-0.207,0.001,0.001,-0.301,0.001
6,"Smith, Will",L,SL,80.3,2328.0,304.0,3.96,2.16,28.991283,80.8,-1.81,1.81,6.55,6.2,54.30,0.69,-0.69,0.83,S,0,2,0-2,-0.152,L,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,59,3,9,0,5,0,5,0,0,0,0,0,45.8,3/4,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.302,0.0,0.196,-0.162,-0.389,-0.123,-0.149,-0.451,-0.149
9,"Gsellman, Robert",R,SI,94.3,1982.0,221.0,16.20,10.56,21.018028,94.4,1.20,-1.20,6.05,6.5,53.98,0.26,-0.26,1.78,X,0,2,0-2,-0.061,R,popup,hit_into_play,field_out,140.0,75.3,65.0,3.0,0.0,1.0,0.000,0.000,0.0,0.0,58,3,8,1,5,0,5,0,0,0,0,2,29.8,overhead,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.302,0.0,0.196,-0.162,-0.260,-0.123,-0.149,-0.451,-0.149
12,"Gsellman, Robert",R,SL,90.8,2052.0,215.0,0.36,5.04,22.599119,90.9,1.48,-1.48,5.95,6.4,54.14,-0.98,0.98,1.12,S,0,2,0-2,-0.101,R,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,57,6,8,1,5,0,5,0,0,0,0,1,27.7,overhead,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.302,0.0,0.196,-0.162,-0.318,-0.123,-0.149,-0.451,-0.149
18,"Gsellman, Robert",R,SL,90.5,2133.0,216.0,-0.12,7.44,23.569061,91.1,1.51,-1.51,5.93,6.3,54.23,-0.01,0.01,2.23,X,1,2,1-2,-0.173,L,line_drive,hit_into_play,field_out,294.0,96.8,18.0,4.0,0.0,1.0,0.409,0.457,0.0,0.0,56,4,8,1,5,0,5,0,0,0,0,0,27.7,overhead,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.302,0.0,0.223,-0.184,-0.411,-0.141,-0.170,-0.472,-0.170
22,"Martin, Chris",R,CH,87.9,1160.0,234.0,7.68,1.80,13.196815,88.8,2.98,-2.98,6.23,6.7,53.81,0.36,-0.36,1.26,S,1,2,1-2,-0.073,L,NaN,strike,field_out,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,55,5,8,0,5,0,5,0,0,0,0,2,42.1,3/4,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.302,0.0,0.223,-0.184,-0.282,-0.141,-0.170,-0.472,-0.170
27,"Martin, Chris",R,FF,94.9,2224.0,216.0,9.00,14.16,23.435195,95.5,2.71,-2.71,6.38,6.6,53.86,0.10,-0.10,1.94,X,0,0,0-0,-0.163,R,fly_ball,hit_into_play,field_out,308.0,91.8,29.0,3.0,0.0,1.0,0.109,0.156,0.0,0.0,54,1,8,0,5,0,5,0,0,0,0,1,40.1,sidearm,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.302,0.0,0.314,-0.043,-0.199,-0.003,-0.004,-0.306,-0.004
28,"Martin, Chris",R,FC,91.1,2156.0,177.0,-3.00,5.28,23.666301,92.1,2.85,-2.85,6.39,6.7,53.80,-0.67,0.67,2.62,X,1,0,1-0,-0.279,L,ground_ball,hit_into_play,field_out,6.0,76.1,-25.0,2.0,0.0,1.0,0.043,0.039,0.0,0.0,53,2,8,0,5,0,5,0,0,0,0,0,42.4,3/4,1,0,1,0,0,0,0,0.481,-0.227,0.254,-0.227,-0.25,0.0,1.209,-0.302,0.0,0.355,-0.051,-0